In [2]:
import pandas as pd
from pathlib import Path
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import LeaveOneOut
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns  # Opcional, para estilização
from sklearn.metrics import explained_variance_score
from sklearn.linear_model import LinearRegression


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\lenau\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\lenau\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\lenau\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\lenau\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\lenau\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\lenau\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

AttributeError: _ARRAY_API not found

In [3]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Definindo o caminho base para a pasta Pre-processamento
base_path = Path(parent_dir) / 'PCA_Palmer'

file_path_raw_cal = base_path / 'RAW_calibration.xlsx'
file_path_msc_cal = base_path / 'MSC_calibration.xlsx'
file_path_snv_cal = base_path / 'SNV_calibration.xlsx'
file_path_sg_cal = base_path / 'SG_calibration.xlsx'

df_raw_cal = pd.read_excel(file_path_raw_cal)
df_msc_cal = pd.read_excel(file_path_msc_cal)
df_snv_cal = pd.read_excel(file_path_snv_cal)
df_sg_cal = pd.read_excel(file_path_sg_cal)

file_path_raw_val = base_path / 'RAW_validation.xlsx'
file_path_msc_val = base_path / 'MSC_validation.xlsx'
file_path_snv_val = base_path / 'SNV_validation.xlsx'
file_path_sg_val = base_path / 'SG_validation.xlsx'

df_raw_val = pd.read_excel(file_path_raw_val)
df_msc_val = pd.read_excel(file_path_msc_val)
df_snv_val = pd.read_excel(file_path_snv_val)
df_sg_val = pd.read_excel(file_path_sg_val)

In [4]:
def choose_optimal_lv(validation, calibration, pls_model, max_lv, X_scaled, Y_scaled):
    """
    Escolhe o número ótimo de componentes latentes (LV) com base nas métricas de calibração e validação cruzada.
    
    Args:
        calibration (pd.DataFrame): DataFrame contendo as métricas de calibração.
        validation (pd.DataFrame): DataFrame contendo as métricas de validação cruzada.
        max_lv (int): Número máximo de LVs a serem considerados.

    Returns:
        int: Número ótimo de LVs.
    """
    lvs = []
    k = 1
    
    for j in range(1, max_lv):
        if j + 1 <= max_lv:
            rpearson_cal_j = calibration.loc[j + 1, 'rpearson_cal']
            rpearson_cv_j = validation.loc[j + 1, 'rpearson_cv']
            rpearson_cal_j_plus_1 = calibration.loc[j, 'rpearson_cal']
            rpearson_cv_j_plus_1 = validation.loc[j, 'rpearson_cv']
            
            # Verifica se há uma melhora em ambas as métricas rpearson (calibração e validação cruzada)
            aux_cal = rpearson_cal_j / rpearson_cal_j_plus_1
            aux_cv = rpearson_cv_j / rpearson_cv_j_plus_1
            
            # Critério: Aumenta o LV se houver uma melhora significativa em ambas as métricas
            if aux_cal > 1 and aux_cv > 1 and ((aux_cv - 1) * 100 >= 2):
                lvs.append(j + 1)
                k += 1

    # Se nenhum LV for selecionado, use o primeiro por padrão
    if len(lvs) == 0:
        return 1

    lv_num = lvs[-1]  # Inicialmente, assume-se o maior LV selecionado

    if len(lvs) > 1:
        pls_model = PLSRegression(n_components=lvs[-2])
        pls_model.fit(X_scaled, Y_scaled)

        # Avalia a estabilidade dos coeficientes do modelo
        for w in range(k - 2):
            if pls_model.coef_.shape[1] > 3:
                if lvs[w + 1] < pls_model.coef_.shape[0] and pls_model.coef_[lvs[w + 1], 3] < 1:
                    lv_num = lvs[w]
                    break

    return lv_num

def train_model(X_scaled, Y_scaled):
    max_lv = 10
    loo = LeaveOneOut()

    calibration = pd.DataFrame(index=range(1, max_lv + 1), columns=['r2_cal', 'neg_root_mean_squared_error_cal', 'explained_variance_cal', 'correlation_cal', 'rpearson_cal', 'secv_cal'])
    validation = pd.DataFrame(index=range(1, max_lv + 1), columns=['r2_cv', 'neg_root_mean_squared_error_cv', 'explained_variance_cv', 'correlation_cv', 'rpearson_cv', 'secv_cv'])
    
    Y_values = pd.DataFrame(index=range(len(Y_scaled)))
    Y_values['Referencia'] = Y_scaled.flatten()  # Valores reais de Y
        
    for lv in range(1, max_lv + 1):
        pls_model = PLSRegression(n_components=lv, max_iter=500)

        # Calibração do modelo
        pls_model.fit(X_scaled, Y_scaled)
        predictions_train = pls_model.predict(X_scaled)

        # Calcula as métricas para a fase de calibração
        calibration.loc[lv, 'r2_cal'] = r2_score(Y_scaled, predictions_train)
        calibration.loc[lv, 'neg_root_mean_squared_error_cal'] = np.sqrt(mean_squared_error(Y_scaled, predictions_train))
        calibration.loc[lv, 'explained_variance_cal'] = explained_variance_score(Y_scaled, predictions_train)

        # Garantir que Y_scaled e predictions_train sejam arrays bidimensionais
        if Y_scaled.ndim == 1:
            Y_scaled = Y_scaled.reshape(-1, 1)
        if predictions_train.ndim == 1:
            predictions_train = predictions_train.reshape(-1, 1)

        correlation_cal = np.corrcoef(Y_scaled[:, 0], predictions_train[:, 0])[0, 1]  # Correlação
        calibration.loc[lv, 'correlation_cal'] = correlation_cal

        # Calcular rpearson para calibração
        rpearson_cal = correlation_cal ** 2
        calibration.loc[lv, 'rpearson_cal'] = rpearson_cal

        # Calcular o viés para calibração
        bias_cal = np.mean(predictions_train - Y_scaled)
        calibration.loc[lv, 'bias_cal'] = bias_cal

        # Calcular SE (Standard Error) para calibração
        sec_cal = np.sqrt(mean_squared_error(Y_scaled, predictions_train))
        calibration.loc[lv, 'sec_cal'] = sec_cal
        
        # Adicionar as previsões de calibração ao DataFrame Y_values
        Y_values[f'Calibracao_LV{lv}'] = predictions_train.flatten()

        # Validação cruzada (Leave-One-Out)
        predictions_cv = cross_val_predict(pls_model, X_scaled, Y_scaled, cv=loo)

        # Calcula as métricas para a validação cruzada
        validation.loc[lv, 'r2_cv'] = r2_score(Y_scaled, predictions_cv)
        validation.loc[lv, 'neg_root_mean_squared_error_cv'] = np.sqrt(mean_squared_error(Y_scaled, predictions_cv))
        validation.loc[lv, 'explained_variance_cv'] = explained_variance_score(Y_scaled, predictions_cv)

        # Garantir que predictions_cv seja array bidimensional
        if predictions_cv.ndim == 1:
            predictions_cv = predictions_cv.reshape(-1, 1)

        correlation_cv = np.corrcoef(Y_scaled[:, 0], predictions_cv[:, 0])[0, 1]  # Correlação
        validation.loc[lv, 'correlation_cv'] = correlation_cv

        # Calcular rpearson para validação cruzada
        rpearson_cv = correlation_cv ** 2
        validation.loc[lv, 'rpearson_cv'] = rpearson_cv

        # Calcular o viés para validação cruzada
        bias_cv = np.mean(predictions_cv - Y_scaled)
        validation.loc[lv, 'bias_cv'] = bias_cv

        # Calcular SECV (Standard Error of Cross Validation)
        secv_cv = np.sqrt(mean_squared_error(Y_scaled, predictions_cv))
        validation.loc[lv, 'secv_cv'] = secv_cv
        
        # Adicionar as previsões de validação ao DataFrame Y_values
        Y_values[f'Validacao_LV{lv}'] = predictions_cv.flatten()
    
    # Escolha o LV ótimo
    optimal_lv = choose_optimal_lv(validation, calibration, pls_model, max_lv, X_scaled, Y_scaled)
    
    # Treina o modelo final com o LV ótimo
    model = PLSRegression(n_components=optimal_lv)
    model.fit(X_scaled, Y_scaled)
    
    return calibration, validation, optimal_lv, model, Y_values


def center_data(X, Y):
    # Calcula as médias para cada coluna
    mx = np.mean(X, axis=0)
    
    # Calcula a média para Y
    my = np.mean(Y)
    
    # Centraliza as colunas na média
    abs_centered = X - mx
    qua_centered = Y - my
       
    return abs_centered, qua_centered, mx, my

def predict(model, X_pred, y_pred, lv_number):
    X_centered, _, _, _ = center_data(X_pred.iloc[:,8:].values, y_pred.values)
    pred = model.predict(X_pred.iloc[:,8:].values)
       
    y_pred = y_pred.values  # Convertendo Series para NumPy array
    pred = pred.reshape(-1, 1)  # Garantindo que pred seja 2D

    if y_pred.ndim == 1:
        y_pred = y_pred.reshape(-1, 1)

    Y_predict = pd.DataFrame(columns=['Amostra', 'Y_pred', 'Y_true', 'Deviation'])
    
    prediction = pd.DataFrame(columns=['r2_cv', 'neg_root_mean_squared_error_cv', 'explained_variance_cv', 'correlation_cv', 'rpearson_cv', 'bias_cv', 'secv_cv'])
    
    prediction.loc[lv_number, 'r2_p'] = r2_score(y_pred, pred)
    prediction.loc[lv_number, 'neg_root_mean_squared_error_p'] = np.sqrt(mean_squared_error(y_pred, pred))
    prediction.loc[lv_number, 'explained_variance_p'] = explained_variance_score(y_pred, pred)
    prediction.loc[lv_number, 'correlation_p'] = np.corrcoef(y_pred[:, 0], pred[:, 0])[0, 1]
    prediction.loc[lv_number, 'rpearson_p'] = np.corrcoef(y_pred[:, 0], pred[:, 0])[0, 1] ** 2  # Quadrado do coeficiente de correlação
    prediction.loc[lv_number, 'bias_p'] = np.mean(pred - y_pred)  # Viés
    prediction.loc[lv_number, 'sec_p'] = np.sqrt(np.mean((pred - y_pred) ** 2))  # Erro quadrático de calibração
    
    Y_predict['Y_pred'] = pred.flatten()
    Y_predict['Y_true'] = y_pred.flatten()
    Y_predict['Deviation'] = pred.flatten() - y_pred.flatten()
    Y_predict['Amostra'] = X_pred.iloc[0:,0].tolist()
        
    return prediction, Y_predict

def plot_pred_vs_ref(Y_true, Y_pred, Y_opt, title="Predicted vs Reference"):
    
    slope_pred, offset_pred = np.polyfit(Y_true, Y_pred, 1)
    rmse_pred = np.sqrt(mean_squared_error(Y_true, Y_pred))
    r2_pred = r2_score(Y_true, Y_pred)
    
    slope_opt, offset_opt = np.polyfit(Y_true, Y_opt, 1)
    rmse_opt = np.sqrt(mean_squared_error(Y_true, Y_opt))
    r2_opt = r2_score(Y_true, Y_opt)
    
    metrics_dict = pd.DataFrame({
    'Y': ['Referência', 'Predição'],
    'R²': [r2_opt, r2_pred],
    'RMSE': [rmse_opt, rmse_pred],
    'Offset': [offset_opt, offset_pred],
    'Slope': [slope_opt,slope_pred]
    })
    
    plt.figure(figsize=(10, 6))
    
     # Scatter plot for reference values
    plt.scatter(Y_true, Y_opt, color='blue', label='Reference', marker='o', alpha=0.7)

    # Adicionar linha de calibração para valores otimizados (Y_opt)
    plt.plot(Y_true, slope_opt * Y_true + offset_opt, color='blue', linestyle='--', label=f'Calibração (Slope={slope_opt:.2f}, Offset={offset_opt:.2f})')

    # Adicionar linha de calibração para valores preditos (Y_pred)
    plt.plot(Y_true, slope_pred * Y_true + offset_pred, color='red', linestyle='--', label=f'Predição (Slope={slope_pred:.2f}, Offset={offset_pred:.2f})')
    
    # Scatter plot for predicted values
    plt.scatter(Y_true, Y_pred, color='red', label='Predicted', marker='o', alpha=0.7)
       
    plt.xlabel("Reference")
    plt.ylabel("Predicted")
    plt.title(title)
    plt.grid(True)
    plt.text(0.05, 0.95, f'Reference - Slope: {slope_opt:.2f}, Offset: {offset_opt:.2f}, RMSE: {rmse_opt:.2f}, R²: {r2_opt:.2f}\nPredicted - Slope: {slope_pred:.2f}, Offset: {offset_pred:.2f}, RMSE: {rmse_pred:.2f}, R²: {r2_pred:.2f}',
             transform=plt.gca().transAxes, fontsize=12, verticalalignment='top', horizontalalignment='left')
    plt.legend(loc='lower right')
    plt.show()
    
    return metrics_dict

def pred_plot(Y_predict):
    X = Y_predict['Y_pred'].values.reshape(-1, 1)
    y = Y_predict['Y_true'].values

    model = LinearRegression().fit(X, y)
    slope = model.coef_[0]
    offset = model.intercept_

    # Criar pontos para a linha de regressão
    X_line = np.array([[Y_predict['Y_pred'].min()], [Y_predict['Y_pred'].max()]])
    y_pred_line = model.predict(X_line)

    r2 = r2_score(y, Y_predict['Y_pred'])
    rmse = np.sqrt(mean_squared_error(y, Y_predict['Y_pred']))

    plt.figure(figsize=(10, 6))
    plt.scatter(Y_predict['Y_pred'], y, color='green', label='Dados')
    plt.plot(X_line, y_pred_line, color='blue', linestyle='--', label='Linha de Regressão')

    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.title('Predições no Conjunto de Teste')
    plt.legend()
    plt.text(0.95, 0.05, f'Slope: {slope:.2f}\nOffset: {offset:.2f}\nR²: {r2:.2f}\nRMSE: {rmse:.2f}', 
         transform=plt.gca().transAxes, verticalalignment='bottom', horizontalalignment='right')
    plt.tight_layout()
    plt.show()
    
    metrics_dict = pd.DataFrame({
    'Y': ['Validação'],
    'R²': [r2],
    'RMSE': [rmse],
    'Offset': [offset],
    'Slope': [slope]
    })
    
    return metrics_dict

def build_model(X_cal, Y_cal, X_val, Y_val):
       
    Y_pred = Y_val
    X_pred = X_val
       
    X = X_cal
    Y = Y_cal
    
    #abs_scaled, qua_scaled, m_abs, m_qua = center_data(X.iloc[:,6:].values, Y.values)
    calibration, validation, lv_num, model, Y_values = train_model(X.iloc[:,8:].values, Y.values)
    print(f'Número de componentes: {lv_num}')
        
    prediction, Y_predict = predict(model, X_pred, Y_pred, lv_num)
    
    Y_true = Y_values.iloc[:, 0]  # Primeira coluna de Y_values
    Y_opt = Y_values[f'Calibracao_LV{lv_num}']  # Coluna específica de calibração
    Y_pred = Y_values[f'Validacao_LV{lv_num}']  # Coluna específica de validação
    plot_infos = plot_pred_vs_ref(Y_true, Y_pred, Y_opt)
    chart_data = pd.DataFrame({
    'Sample': Y_values.index,  # Identificador da amostra, pode ser index ou outra coluna específica
    'Y_true': Y_true,
    'Y_opt': Y_opt,
    'Y_pred': Y_pred
    })  
    #display(chart_data)
    pred_plot(Y_predict)
    #display(Y_predict)
    plot_infos_p = pred_plot(Y_predict)
    #display(Y_predict)
    plot_infos_df = plot_infos.copy()
    
    # Adiciona a nova linha ao DataFrame existente
    plot_infos_pre = pd.concat([plot_infos_df, plot_infos_p], ignore_index=True)
    
    return calibration, validation, Y_predict, plot_infos_pre


In [5]:
plot_infos_msc = pd.DataFrame(columns=['Attribute', 'Y', 'R²', 'RMSE','Offset','Slope'])
plot_infos_snv = pd.DataFrame(columns=['Attribute', 'Y', 'R²', 'RMSE','Offset','Slope'])
plot_infos_sg = pd.DataFrame(columns=['Attribute', 'Y', 'R²', 'RMSE','Offset','Slope'])
plot_infos_raw = pd.DataFrame(columns=['Attribute', 'Y', 'R²', 'RMSE','Offset','Slope'])

def add_plot_infos(plot_infos_pre, plot_infos_df, attribute):
    # Cria uma cópia do DataFrame para evitar alterações no original
    plot_infos_df = plot_infos_df.copy()
    
    # Adiciona a coluna 'Attribute' ao DataFrame plot_infos_df
    plot_infos_df['Attribute'] = attribute
    
    # Adiciona a nova linha ao DataFrame existente
    plot_infos_pre = pd.concat([plot_infos_pre, plot_infos_df], ignore_index=True)
    
    return plot_infos_pre

In [6]:
print(df_raw_cal.columns)

Index([   'Amostra',        'SST',    'FIRMEZA',        'ATT',        'AAC',
       'MASSA SECA',   900.577121,   904.475237,   908.369055,   912.258573,
       ...
        1675.187331,  1678.109677,  1681.027723,  1683.941471,  1686.850921,
        1689.756072,  1692.656924,  1694.588437,  1697.482126,  1700.371515],
      dtype='object', length=254)


In [7]:
calibration_msc_sst, validation_msc_sst, Y_predict_msc_sst, plot_infos = build_model(df_msc_cal, df_msc_cal['SST'], df_msc_val, df_msc_val['SST'])

plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'SST')

Número de componentes: 7


In [8]:
calibration_msc_firmeza, validation_msc_firmeza, Y_predict_msc_firmeza, plot_infos = build_model(df_msc_cal, df_msc_cal['FIRMEZA'], df_msc_val, df_msc_val['FIRMEZA'])

plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'FIRMEZA')

Número de componentes: 10


In [9]:
calibration_msc_aac, validation_msc_aac, Y_predict_msc_aac, plot_infos = build_model(df_msc_cal, df_msc_cal['AAC'], df_msc_val, df_msc_val['AAC'])

plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'AAC')

Número de componentes: 10


In [10]:
calibration_msc_att, validation_msc_att, Y_predict_msc_att, plot_infos = build_model(df_msc_cal, df_msc_cal['ATT'], df_msc_val, df_msc_val['ATT'])

plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'ATT')

Número de componentes: 7


In [11]:
calibration_msc_ubs, validation_msc_ubs, Y_predict_msc_ubs, plot_infos = build_model(df_msc_cal, df_msc_cal['MASSA SECA'], df_msc_val, df_msc_val['MASSA SECA'])

plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'MASSA SECA')

Número de componentes: 10


In [12]:
plot_infos_msc.to_excel('plot_infos_plsr_msc.xlsx', index=False)
plot_infos_msc

,Attribute,Y,R²,RMSE,Offset,Slope
0,SST,Referência,0.684174,1.619691,2.553681,0.684174
1,SST,Predição,0.207256,2.566106,4.169829,0.496397
2,SST,Validação,0.582246,1.683444,1.633198,0.849785
3,FIRMEZA,Referência,0.955340,9.662608,2.602729,0.955340
4,FIRMEZA,Predição,0.137707,42.458211,23.952340,0.583287
5,FIRMEZA,Validação,0.518288,27.503669,-0.217463,0.833218
6,AAC,Referência,0.891311,2.186481,3.159181,0.891311
7,AAC,Predição,-0.159726,7.142190,20.925841,0.291131
8,AAC,Validação,-0.178308,4.868276,16.918602,0.442592
9,ATT,Referência,0.638488,0.089115,0.140714,0.638488


In [13]:
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# Usar um backend não interativo adequado para salvar gráficos
import matplotlib
matplotlib.use('Agg')

# Diretório onde os gráficos serão salvos
GRAPH_FOLDER = 'graphs_msc'
if not os.path.exists(GRAPH_FOLDER):
    os.makedirs(GRAPH_FOLDER)

# Função para salvar gráficos corretamente
def save_plot(title):
    filename = f"{title.replace(' ', '_').lower()}.png"
    filepath = os.path.join(GRAPH_FOLDER, filename)
    
    # Forçar o gráfico a ser desenhado completamente antes de salvar
    plt.draw()
    
    # Salvar o gráfico com 'bbox_inches' para garantir que não haja cortes
    plt.savefig(filepath, bbox_inches='tight')
    print(f"Gráfico salvo em: {filepath}")
    
    # Fechar o gráfico para evitar sobreposição
    plt.close()

# Função para gerar o gráfico de "Predições no Conjunto de Teste" e salvar
def pred_plot(Y_predict, title="Predicoes_no_Conjunto_de_Teste"):
    X = Y_predict['Y_pred'].values.reshape(-1, 1)
    y = Y_predict['Y_true'].values

    model = LinearRegression().fit(X, y)
    slope = model.coef_[0]
    offset = model.intercept_

    # Criar pontos para a linha de regressão
    X_line = np.array([[Y_predict['Y_pred'].min()], [Y_predict['Y_pred'].max()]])
    y_pred_line = model.predict(X_line)

    r2 = r2_score(y, Y_predict['Y_pred'])
    rmse = np.sqrt(mean_squared_error(y, Y_predict['Y_pred']))

    # Geração do gráfico
    plt.figure(figsize=(10, 6))
    plt.scatter(Y_predict['Y_pred'], y, color='green', label='Dados')
    plt.plot(X_line, y_pred_line, color='blue', linestyle='--', label='Linha de Regressão')

    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.title('Predições no Conjunto de Teste')
    plt.legend()
    plt.text(0.95, 0.05, f'Slope: {slope:.2f}\nOffset: {offset:.2f}\nR²: {r2:.2f}\nRMSE: {rmse:.2f}', 
             transform=plt.gca().transAxes, verticalalignment='bottom', horizontalalignment='right')
    plt.tight_layout()
    
    # Salvar o gráfico
    save_plot(title)

    metrics_dict = pd.DataFrame({
        'Y': ['Validação'],
        'R²': [r2],
        'RMSE': [rmse],
        'Offset': [offset],
        'Slope': [slope]
    })
    
    #return metrics_dict


# Exemplo de chamada para o gráfico de SST
# Primeiro gráfico: Predicted vs Reference
plt.figure()  # Inicia uma nova figura
calibration_msc_sst, validation_msc_sst, Y_predict_msc_sst, plot_infos = build_model(df_msc_cal, df_msc_cal['SST'], df_msc_val, df_msc_val['SST'])
plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'SST')
save_plot("SST_Predicted_vs_Reference")  # Salva o gráfico de "Predicted vs Reference"


# Segundo gráfico: Predições no Conjunto de Teste
pred_plot(Y_predict_msc_sst, "SST_Predicoes_no_Conjunto_de_Teste")  # Salva o gráfico de "Predições no Conjunto de Teste"


# Repita o mesmo para os outros parâmetros
# FIRMEZA
plt.figure()
calibration_msc_firmeza, validation_msc_firmeza, Y_predict_msc_firmeza, plot_infos = build_model(df_msc_cal, df_msc_cal['FIRMEZA'], df_msc_val, df_msc_val['FIRMEZA'])
plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'FIRMEZA')
save_plot("Firmeza_Predicted_vs_Reference")
pred_plot(Y_predict_msc_firmeza, "Firmeza_Predicoes_no_Conjunto_de_Teste")

# AAC
plt.figure()
calibration_msc_aac, validation_msc_aac, Y_predict_msc_aac, plot_infos = build_model(df_msc_cal, df_msc_cal['AAC'], df_msc_val, df_msc_val['AAC'])
plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'AAC')
save_plot("AAC_Predicted_vs_Reference")
pred_plot(Y_predict_msc_aac, "AAC_Predicoes_no_Conjunto_de_Teste")

# ATT
plt.figure()
calibration_msc_att, validation_msc_att, Y_predict_msc_att, plot_infos = build_model(df_msc_cal, df_msc_cal['ATT'], df_msc_val, df_msc_val['ATT'])
plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'ATT')
save_plot("ATT_Predicted_vs_Reference")
pred_plot(Y_predict_msc_att, "ATT_Predicoes_no_Conjunto_de_Teste")

# MASSA SECA
plt.figure()
calibration_msc_ubs, validation_msc_ubs, Y_predict_msc_ubs, plot_infos = build_model(df_msc_cal, df_msc_cal['MASSA SECA'], df_msc_val, df_msc_val['MASSA SECA'])
plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'MASSA SECA')
save_plot("Massa_Seca_Predicted_vs_Reference")
pred_plot(Y_predict_msc_ubs, "Massa_Seca_Predicoes_no_Conjunto_de_Teste")


Número de componentes: 7
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\sst_predicted_vs_reference.png
Gráfico salvo em: graphs_msc\sst_predicoes_no_conjunto_de_teste.png
Número de componentes: 10
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\firmeza_predicted_vs_reference.png
Gráfico salvo em: graphs_msc\firmeza_predicoes_no_conjunto_de_teste.png
Número de componentes: 10
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\aac_predicted_vs_reference.png
Gráfico salvo em: graphs_msc\aac_predicoes_no_conjunto_de_teste.png
Número de componentes: 7
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_test

In [14]:
calibration_snv_sst, validation_snv_sst, Y_predict_snv_sst, plot_infos = build_model(df_snv_cal, df_snv_cal['SST'], df_snv_val, df_snv_val['SST'])

plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'SST')

Número de componentes: 9
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png


In [15]:
calibration_snv_firmeza, validation_snv_firmeza, Y_predict_snv_firmeza, plot_infos = build_model(df_snv_cal, df_snv_cal['FIRMEZA'], df_snv_val, df_snv_val['FIRMEZA'])

plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'FIRMEZA')

Número de componentes: 10
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png


In [16]:
calibration_snv_aac, validation_snv_aac, Y_predict_snv_aac, plot_infos = build_model(df_snv_cal, df_snv_cal['AAC'], df_snv_val, df_snv_val['AAC'])

plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'AAC')

Número de componentes: 10
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png


In [17]:
calibration_snv_att, validation_snv_att, Y_predict_snv_att, plot_infos = build_model(df_snv_cal, df_snv_cal['ATT'], df_snv_val, df_snv_val['ATT'])

plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'ATT')

Número de componentes: 9
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png


In [18]:
calibration_snv_ubs, validation_snv_ubs, Y_predict_snv_ubs, plot_infos = build_model(df_snv_cal, df_snv_cal['MASSA SECA'], df_snv_val, df_snv_val['MASSA SECA'])

plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'MASSA SECA')

Número de componentes: 7
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_msc\predicoes_no_conjunto_de_teste.png


In [19]:
plot_infos_snv.to_excel('plot_infos_plsr_snv.xlsx', index=False)
plot_infos_snv

,Attribute,Y,R²,RMSE,Offset,Slope
0,SST,Referência,0.876957,1.003375,1.000746,0.876957
1,SST,Predição,0.159291,2.622760,4.286781,0.468898
2,FIRMEZA,Referência,0.948526,10.369358,2.998650,0.948526
3,FIRMEZA,Predição,0.064653,44.202365,26.762133,0.559437
4,AAC,Referência,0.892733,2.166185,3.128828,0.892733
5,AAC,Predição,-0.176413,7.173684,21.231322,0.284366
6,ATT,Referência,0.841414,0.055683,0.060966,0.841414
7,ATT,Predição,-0.020614,0.141261,0.243993,0.342590
8,MASSA SECA,Referência,0.654503,0.015221,0.045282,0.654503
9,MASSA SECA,Predição,-0.056173,0.026612,0.079258,0.397957


In [29]:
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# Usar um backend não interativo adequado para salvar gráficos
import matplotlib
matplotlib.use('Agg')

# Diretório onde os gráficos serão salvos
GRAPH_FOLDER = 'graphs_snv'
if not os.path.exists(GRAPH_FOLDER):
    os.makedirs(GRAPH_FOLDER)

# Função para salvar gráficos corretamente
def save_plot(title):
    filename = f"{title.replace(' ', '_').lower()}.png"
    filepath = os.path.join(GRAPH_FOLDER, filename)
    
    # Forçar o gráfico a ser desenhado completamente antes de salvar
    plt.draw()
    
    # Salvar o gráfico com 'bbox_inches' para garantir que não haja cortes
    plt.savefig(filepath, bbox_inches='tight')
    print(f"Gráfico salvo em: {filepath}")
    
    # Fechar o gráfico para evitar sobreposição
    plt.close()

# Função para gerar o gráfico de "Predições no Conjunto de Teste" e salvar
def pred_plot(Y_predict, title="Predicoes_no_Conjunto_de_Teste"):
    X = Y_predict['Y_pred'].values.reshape(-1, 1)
    y = Y_predict['Y_true'].values

    model = LinearRegression().fit(X, y)
    slope = model.coef_[0]
    offset = model.intercept_

    # Criar pontos para a linha de regressão
    X_line = np.array([[Y_predict['Y_pred'].min()], [Y_predict['Y_pred'].max()]])
    y_pred_line = model.predict(X_line)

    r2 = r2_score(y, Y_predict['Y_pred'])
    rmse = np.sqrt(mean_squared_error(y, Y_predict['Y_pred']))

    # Geração do gráfico
    plt.figure(figsize=(10, 6))
    plt.scatter(Y_predict['Y_pred'], y, color='green', label='Dados')
    plt.plot(X_line, y_pred_line, color='blue', linestyle='--', label='Linha de Regressão')

    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.title('Predições no Conjunto de Teste')
    plt.legend()
    plt.text(0.95, 0.05, f'Slope: {slope:.2f}\nOffset: {offset:.2f}\nR²: {r2:.2f}\nRMSE: {rmse:.2f}', 
             transform=plt.gca().transAxes, verticalalignment='bottom', horizontalalignment='right')
    plt.tight_layout()
    
    # Salvar o gráfico
    save_plot(title)

    metrics_dict = pd.DataFrame({
        'Y': ['Validação'],
        'R²': [r2],
        'RMSE': [rmse],
        'Offset': [offset],
        'Slope': [slope]
    })
    
    #return metrics_dict


# Exemplo de chamada para o gráfico de SST
# Primeiro gráfico: Predicted vs Reference
plt.figure()  # Inicia uma nova figura
calibration_snv_sst, validation_snv_sst, Y_predict_snv_sst, plot_infos = build_model(df_snv_cal, df_snv_cal['SST'], df_snv_val, df_snv_val['SST'])
plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'SST')
save_plot("SST_Predicted_vs_Reference")  # Salva o gráfico de "Predicted vs Reference"
# Segundo gráfico: Predições no Conjunto de Teste
pred_plot(Y_predict_snv_sst, "SST_Predicoes_no_Conjunto_de_Teste")  # Salva o gráfico de "Predições no Conjunto de Teste"


# Repita o mesmo para os outros parâmetros
# FIRMEZA
plt.figure()
calibration_snv_firmeza, validation_snv_firmeza, Y_predict_snv_firmeza, plot_infos = build_model(df_snv_cal, df_snv_cal['FIRMEZA'], df_snv_val, df_snv_val['FIRMEZA'])
plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'FIRMEZA')
save_plot("Firmeza_Predicted_vs_Reference")
pred_plot(Y_predict_snv_firmeza, "Firmeza_Predicoes_no_Conjunto_de_Teste")

# AAC
plt.figure()
calibration_snv_aac, validation_snv_aac, Y_predict_snv_aac, plot_infos = build_model(df_snv_cal, df_snv_cal['AAC'], df_snv_val, df_snv_val['AAC'])
plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'AAC')
save_plot("AAC_Predicted_vs_Reference")
pred_plot(Y_predict_snv_aac, "AAC_Predicoes_no_Conjunto_de_Teste")

# ATT
plt.figure()
calibration_snv_att, validation_snv_att, Y_predict_snv_att, plot_infos = build_model(df_snv_cal, df_snv_cal['ATT'], df_snv_val, df_snv_val['ATT'])
plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'ATT')
save_plot("ATT_Predicted_vs_Reference")
pred_plot(Y_predict_snv_att, "ATT_Predicoes_no_Conjunto_de_Teste")

# MASSA SECA
plt.figure()
calibration_snv_ubs, validation_snv_ubs, Y_predict_snv_ubs, plot_infos = build_model(df_snv_cal, df_snv_cal['MASSA SECA'], df_snv_val, df_snv_val['MASSA SECA'])
plot_infos_snv = add_plot_infos(plot_infos_snv, plot_infos, 'MASSA SECA')
save_plot("Massa_Seca_Predicted_vs_Reference")
pred_plot(Y_predict_snv_ubs, "Massa_Seca_Predicoes_no_Conjunto_de_Teste")

Número de componentes: 9


NameError: name 'Y_true' is not defined

In [21]:
calibration_sg_sst, validation_sg_sst, Y_predict_sg_sst, plot_infos = build_model(df_sg_cal, df_sg_cal['SST'], df_sg_val, df_sg_val['SST'])

plot_infos_sg = add_plot_infos(plot_infos_sg, plot_infos, 'SST')




Número de componentes: 10
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png


In [22]:
calibration_sg_firmeza, validation_sg_firmeza, Y_predict_sg_firmeza, plot_infos = build_model(df_sg_cal, df_sg_cal['FIRMEZA'], df_sg_val, df_sg_val['FIRMEZA'])

plot_infos_sg = add_plot_infos(plot_infos_sg, plot_infos, 'FIRMEZA')

Número de componentes: 9
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png


In [23]:
calibration_sg_aac, validation_sg_aac, Y_predict_sg_aac, plot_infos = build_model(df_sg_cal, df_sg_cal['AAC'], df_sg_val, df_sg_val['AAC'])

plot_infos_sg = add_plot_infos(plot_infos_sg, plot_infos, 'AAC')

Número de componentes: 9
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png


In [24]:
calibration_sg_att, validation_sg_att, Y_predict_sg_att, plot_infos = build_model(df_sg_cal, df_sg_cal['ATT'], df_sg_val, df_sg_val['ATT'])

plot_infos_sg = add_plot_infos(plot_infos_sg, plot_infos, 'ATT')

Número de componentes: 10
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png


In [25]:
calibration_sg_ubs, validation_sg_ubs, Y_predict_sg_ubs, plot_infos = build_model(df_sg_cal, df_sg_cal['MASSA SECA'], df_sg_val, df_sg_val['MASSA SECA'])

plot_infos_sg = add_plot_infos(plot_infos_sg, plot_infos, 'MASSA SECA')

Número de componentes: 7
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png
Gráfico salvo em: graphs_snv\predicoes_no_conjunto_de_teste.png


In [26]:
plot_infos_sg.to_excel('plot_infos_plsr_sg.xlsx', index=False)
plot_infos_sg

,Attribute,Y,R²,RMSE,Offset,Slope
0,SST,Referência,0.643420,1.721022,2.883203,0.643420
1,SST,Predição,0.295097,2.419762,3.977522,0.512136
2,FIRMEZA,Referência,0.822587,19.258685,10.339357,0.822587
3,FIRMEZA,Predição,0.618953,28.224310,13.451478,0.750433
4,AAC,Referência,0.436656,4.977835,16.374345,0.436656
5,AAC,Predição,-0.118475,7.014017,22.823493,0.225347
6,ATT,Referência,0.697500,0.081518,0.117744,0.697500
7,ATT,Predição,0.371609,0.117491,0.156167,0.591014
8,MASSA SECA,Referência,0.468982,0.019034,0.069015,0.468982
9,MASSA SECA,Predição,-0.067388,0.026986,0.091697,0.286527


In [28]:
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

# Usar um backend não interativo adequado para salvar gráficos
import matplotlib
matplotlib.use('Agg')

# Diretório onde os gráficos serão salvos
GRAPH_FOLDER = 'graphs_sg'
if not os.path.exists(GRAPH_FOLDER):
    os.makedirs(GRAPH_FOLDER)

# Função para salvar gráficos corretamente
def save_plot(title):
    filename = f"{title.replace(' ', '_').lower()}.png"
    filepath = os.path.join(GRAPH_FOLDER, filename)
    
    # Forçar o gráfico a ser desenhado completamente antes de salvar
    plt.draw()
    
    # Salvar o gráfico com 'bbox_inches' para garantir que não haja cortes
    plt.savefig(filepath, bbox_inches='tight')
    print(f"Gráfico salvo em: {filepath}")
    
    # Fechar o gráfico para evitar sobreposição
    plt.close()

# Função para gerar o gráfico de "Predições no Conjunto de Teste" e salvar
def pred_plot(Y_predict, title="Predicoes_no_Conjunto_de_Teste"):
    X = Y_predict['Y_pred'].values.reshape(-1, 1)
    y = Y_predict['Y_true'].values

    model = LinearRegression().fit(X, y)
    slope = model.coef_[0]
    offset = model.intercept_

    # Criar pontos para a linha de regressão
    X_line = np.array([[Y_predict['Y_pred'].min()], [Y_predict['Y_pred'].max()]])
    y_pred_line = model.predict(X_line)

    r2 = r2_score(y, Y_predict['Y_pred'])
    rmse = np.sqrt(mean_squared_error(y, Y_predict['Y_pred']))

    # Geração do gráfico
    plt.figure(figsize=(10, 6))
    plt.scatter(Y_predict['Y_pred'], y, color='green', label='Dados')
    plt.plot(X_line, y_pred_line, color='blue', linestyle='--', label='Linha de Regressão')

    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.title('Predições no Conjunto de Teste')
    plt.legend()
    plt.text(0.95, 0.05, f'Slope: {slope:.2f}\nOffset: {offset:.2f}\nR²: {r2:.2f}\nRMSE: {rmse:.2f}', 
             transform=plt.gca().transAxes, verticalalignment='bottom', horizontalalignment='right')
    plt.tight_layout()
    
    # Salvar o gráfico
    save_plot(title)

    metrics_dict = pd.DataFrame({
        'Y': ['Validação'],
        'R²': [r2],
        'RMSE': [rmse],
        'Offset': [offset],
        'Slope': [slope]
    })
    
    #return metrics_dict


# Exemplo de chamada para o gráfico de SST
# Primeiro gráfico: Predicted vs Reference
plt.figure()  # Inicia uma nova figura
calibration_sg_sst, validation_sg_sst, Y_predict_sg_sst, plot_infos = build_model(df_sg_cal, df_sg_cal['SST'], df_sg_val, df_sg_val['SST'])
plot_infos_sg = add_plot_infos(plot_infos_sg, plot_infos, 'SST')
save_plot("SST_Predicted_vs_Reference")  # Salva o gráfico de "Predicted vs Reference"
# Segundo gráfico: Predições no Conjunto de Teste
pred_plot(Y_predict_sg_sst, "SST_Predicoes_no_Conjunto_de_Teste")  # Salva o gráfico de "Predições no Conjunto de Teste"


# Repita o mesmo para os outros parâmetros
# FIRMEZA
plt.figure()
calibration_msc_firmeza, validation_msc_firmeza, Y_predict_msc_firmeza, plot_infos = build_model(df_msc_cal, df_msc_cal['FIRMEZA'], df_msc_val, df_msc_val['FIRMEZA'])
plot_infos_msc = add_plot_infos(plot_infos_msc, plot_infos, 'FIRMEZA')
save_plot("Firmeza_Predicted_vs_Reference")
pred_plot(Y_predict_sg_firmeza, "Firmeza_Predicoes_no_Conjunto_de_Teste")

# AAC
plt.figure()
calibration_sg_aac, validation_sg_aac, Y_predict_sg_aac, plot_infos = build_model(df_sg_cal, df_sg_cal['AAC'], df_sg_val, df_sg_val['AAC'])
plot_infos_sg = add_plot_infos(plot_infos_sg, plot_infos, 'AAC')
save_plot("AAC_Predicted_vs_Reference")
pred_plot(Y_predict_sg_aac, "AAC_Predicoes_no_Conjunto_de_Teste")

# ATT
plt.figure()
calibration_sg_att, validation_sg_att, Y_predict_sg_att, plot_infos = build_model(df_sg_cal, df_sg_cal['ATT'], df_sg_val, df_sg_val['ATT'])
plot_infos_sg = add_plot_infos(plot_infos_sg, plot_infos, 'ATT')
save_plot("ATT_Predicted_vs_Reference")
pred_plot(Y_predict_sg_att, "ATT_Predicoes_no_Conjunto_de_Teste")

# MASSA SECA
plt.figure()
calibration_sg_ubs, validation_sg_ubs, Y_predict_sg_ubs, plot_infos = build_model(df_sg_cal, df_sg_cal['MASSA SECA'], df_sg_val, df_sg_val['MASSA SECA'])
plot_infos_sg = add_plot_infos(plot_infos_sg, plot_infos, 'MASSA SECA')
save_plot("Massa_Seca_Predicted_vs_Reference")
pred_plot(Y_predict_sg_ubs, "Massa_Seca_Predicoes_no_Conjunto_de_Teste")


Número de componentes: 10


NameError: name 'Y_true' is not defined